<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')

In [7]:
# '일시' 열을 날짜 데이터로 변환
data['일시'] = pd.to_datetime(data['일시'])

# 2월 29일에 해당하는 행 선택
selected_rows = data[(data['일시'].dt.month == 2) & (data['일시'].dt.day == 29)]

# 29일에 해당하는 16개 행
# 23011 - 16 = 22995

# 선택한 행의 인덱스를 사용하여 삭제
data = data.drop(selected_rows.index)

In [8]:
# 계절 분류 함수 정의
def classify_season(month):
    if  month == 1 or month == 2 or month == 12:
        return '겨울'
    elif 3 <= month <= 5:
        return '봄'
    elif 6 <= month <= 8:
        return '여름'
    else:
        return '가을'

# '일시' 컬럼에서 월 정보를 추출하여 '계절' 열 추가
data['월'] = data['일시'].dt.month
data['계절'] = data['월'].apply(classify_season)

# 계절별 중앙값 계산
seasonal_medians = data.groupby('계절').transform('median')

# 결측값 대체
data.fillna(seasonal_medians, inplace=True)

<ipython-input-8-2bcbecf70c8d>:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seasonal_medians = data.groupby('계절').transform('median')


In [9]:
train = data[['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율']]

target = data['평균기온']

In [10]:
data['연도'] = data['일시'].dt.year

data_after_1980 = data[data['연도'] >= 1980]

train_80 = data_after_1980[['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율']]
target_80 = data_after_1980['평균기온']

In [11]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(train_80, target_80)

pred_rf = rf.predict(train_80)

from sklearn.metrics import mean_absolute_error
rf_mae = mean_absolute_error(target_80, pred_rf)
print(rf_mae)

submission['평균기온'] = pred_rf[:358]
print(submission.head(30))

0.13898075820324948
            일시    평균기온
0   2023-01-01   3.731
1   2023-01-02   4.678
2   2023-01-03   3.500
3   2023-01-04  -3.386
4   2023-01-05  -5.311
5   2023-01-06  -2.675
6   2023-01-07  -8.541
7   2023-01-08  -4.365
8   2023-01-09   0.053
9   2023-01-10  -7.841
10  2023-01-11  -8.477
11  2023-01-12  -3.972
12  2023-01-13  -3.310
13  2023-01-14  -5.698
14  2023-01-15  -5.183
15  2023-01-16  -6.205
16  2023-01-17 -12.384
17  2023-01-18 -10.133
18  2023-01-19  -4.320
19  2023-01-20  -5.130
20  2023-01-21  -7.267
21  2023-01-22  -7.758
22  2023-01-23  -0.756
23  2023-01-24  -2.570
24  2023-01-25  -2.713
25  2023-01-26   1.625
26  2023-01-27   5.269
27  2023-01-28   5.233
28  2023-01-29   3.159
29  2023-01-30  -9.542


In [12]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()
poly.fit(train_80)
train_poly = poly.transform(train_80)

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_poly, target_80)

# print(lr.score(train_poly, target))

from sklearn.metrics import mean_absolute_error
lr_pred = lr.predict(train_poly)

lr_mae = mean_absolute_error(target_80, lr_pred)

print("훈련 세트 MAE:", lr_mae)

submission['평균기온'] = lr_pred[:358]
# submission.to_csv("DEAGYUL_9_submission.csv", index=False)

print(submission.head(30))

훈련 세트 MAE: 0.3559908563300467
            일시       평균기온
0   2023-01-01   3.626416
1   2023-01-02   4.615129
2   2023-01-03   2.608891
3   2023-01-04  -3.852668
4   2023-01-05  -5.307220
5   2023-01-06  -2.203420
6   2023-01-07  -8.745922
7   2023-01-08  -4.328595
8   2023-01-09  -0.355605
9   2023-01-10  -7.094057
10  2023-01-11  -8.012096
11  2023-01-12  -3.902322
12  2023-01-13  -3.486952
13  2023-01-14  -5.463948
14  2023-01-15  -4.967372
15  2023-01-16  -5.838511
16  2023-01-17 -12.220932
17  2023-01-18  -9.933655
18  2023-01-19  -4.231407
19  2023-01-20  -5.255504
20  2023-01-21  -6.850748
21  2023-01-22  -7.590851
22  2023-01-23  -0.070325
23  2023-01-24  -2.313619
24  2023-01-25  -2.576469
25  2023-01-26   1.524478
26  2023-01-27   5.801514
27  2023-01-28   5.259564
28  2023-01-29   2.453878
29  2023-01-30  -7.891416


In [13]:
submission.to_csv("DEAGYUL_22_submission.csv", index=False)

여기서는 점수가 약상승하는 반면 실제 제출점수는 크게 떨어졌다

80년대 이후 데이터가 선형 회귀에 우상향하게 나와서 더 적합하다고 판단했지만 데이터를 줄일수록 채점 점수가 계속 떨어졌다